# 1. INICIO

Nuestro punto de partida son los Data Sets provistos por [Buenos Aires Data](https://data.buenosaires.gob.ar/dataset/bicicletas-publicas), los cuales tienen caracteristicas similares, pero presentan algunas discrepancias que dificultan hacer un análisis para todos los años, por lo que nuestros objetivos serán:

- Definir el nombre generico para las columnas.
- Definir las columnas compatibles.
- Unificar los data sets en un **NetCDF**

In [1]:
from localpacks.data import get_data

#### 1.1  Importamos nuestros datasets

In [2]:
#Rango de 2010 al 2018
df_2010 = get_data(year=2010)
df_2011 = get_data(year=2011)
df_2012 = get_data(year=2012)
df_2013 = get_data(year=2013)
df_2014 = get_data(year=2014)
df_2015 = get_data(year=2015)
df_2016 = get_data(year=2016)
df_2017 = get_data(year=2017)
df_2018 = get_data(year=2018) #Aun no esta completo fecha_actual: 13/12/2018

In [3]:
#observamos las columnas de cada data frame
df_2010.columns.values
#df_2011.columns.values
#df_2012.columns.values
#df_2013.columns.values
#df_2014.columns.values
#df_2015.columns.values
#df_2016.columns.values
#df_2017.columns.values
#df_2018.columns.values

array(['PERIODO', 'ORIGENFECHA', 'ORIGENESTACIONID', 'ORIGENNOMBRE',
       'DESTINOFECHA', 'DESTINOESTACIONID', 'DESTINONOMBRE', 'TIEMPOUSO'],
      dtype=object)

#### 1.2 Extraemos las columnas y las vaciamos en un dict

In [4]:
#columnas iniciales
columns = {'2010' : ['PERIODO', 'ORIGENFECHA', 'ORIGENESTACIONID', 'ORIGENNOMBRE',
       'DESTINOFECHA', 'DESTINOESTACIONID', 'DESTINONOMBRE', 'TIEMPOUSO'],

           '2011' : ['PERIODO', 'ORIGENFECHA', 'ORIGENESTACIONID', 'NOMBREORIGEN',
       'DESTINOFECHA', 'DESTINOESTACIONID', 'NOMBREDESTINO', 'TIEMPOUSO'],
           
           '2012' : ['PERIODO', 'ORIGENFECHA', 'ORIGENESTACIONID', 'ORIGENNOMBRE',
       'DESTINOFECHA', 'DESTINOESTACIONID', 'DESTINONOMBRE', 'TIEMPOUSO'],
           
           '2013' : ['ID', 'NOMBRE_ORIGEN', 'ORIGEN_FECHA', 'DESTINO_ESTACION',
       'DESTINO_FECHA'],
           
           '2014' : ['ID', 'NOMBRE_ORIGEN', 'ORIGEN_FECHA', 'DESTINO_ESTACION',
       'DESTINO_FECHA'],
           
           '2015' : ['periodo', 'genero', 'fecha_hora_retiro', 'origen_estacion',
       'nombre_origen', 'destino_estacion', 'nombre_destino', 'tiempo_uso'],
           
           '2016' : ['periodo', 'genero', 'fecha_hora_retiro', 'origen_estacion',
       'nombre_origen', 'destino_estacion', 'nombre_destino', 'tiempo_uso'],
           
           '2017' : ['periodo', 'genero', 'fecha_hora_retiro', 'origen_estacion',
       'nombre_origen', 'destino_estacion', 'nombre_destino', 'tiempo_uso'],
           
           '2018' : ['id_usuario', 'bici_fecha_hora_retiro', 'bici_tiempo_uso', 
                     'bici_nombre_estacion_origen', 'bici_estacion_origen',
                     'bici_nombre_estacion_destino', 'bici_estacion_destino',
                     'bici_sexo', 'bici_edad']
          }

Con el analisis de los nombres de cada columna, definimos las siguientes convenciones:

#### 1.3 Definicion de nombres

2010 al 2012 | 2013 al 2014 | 2015 al 2017 | 2018
------------ | ------------- | ------------ | -------------
periodo | -- | periodo | --
-- | usuario_id | -- | usuario_id
origen_fecha, origen_id, origen_nombre | origen_fecha, origen_id, origen_nombre | origen_fecha, origen_id, origen_nombre | origen_fecha, origen_id, origen_nombre
destino_fecha, destino_id, destino_nombre| destino_fecha, destino_id, destino_nombre | destino_id, destino_nombre | destino_id, destino_nombre
tiempo_uso | -- | tiempo_uso | tiempo_uso
-- | -- | -- | usario_edad
-- | -- | usuario_genero | usuario_genero


En la tabla anterior dejamos claro el nombre que utilizaremos para las columnas comunes y no comunes.

#### 1.4 Formato de integración

2010 al 2012 | 2013 al 2014 | 2015 al 2017 | 2018
------------ | ------------- | ------------ | -------------
-- | usuario_id | -- | usuario_id
fecha_uso | fecha_uso | fecha_uso | fecha_uso 
origen_id, origen_nombre | origen_id, origen_nombre | origen_id, origen_nombre | origen_id, origen_nombre
destino_id, destino_nombre| destino_id, destino_nombre | destino_id, destino_nombre | destino_id, destino_nombre
tiempo_uso(s) | tiempo_uso(s) | tiempo_uso(s) | tiempo_uso(s)
-- | -- | -- | usario_edad
-- | -- | usuario_genero | usuario_genero


Consideraciones:

- La columna **periodo** se elimina, y pasa a ser un metadato.
- La columna **origen fecha** se cambia a **fecha_uso** y pasa a ser el **index**
- La culumna **destino fecha** si existe se elimina y se deriva el **tiempo_uso**

Formatos:

- **usuario_id:**  integer, discreto, ordinal.
- **fecha_uso:** date_time, discreto, ordinal.
- **origen_id:** integer, discreto, ordinal (one_to_one with origen_nombre)
- **nombre_id:** string, discreto, ordinal (one_to_one with origen_id)
- **destino_id:**  integer, discreto, ordinal (one_to_one with destino_nombre)
- **destino_id:**  string, discreto, ordinal (one_to_one with destino_id)
- **tiempo_uso:** time_delta, continuo, ordinal.
- **usuario_edad:** integer, discreto, ordinal.
- **usuario_genero:** string.

# 2. AJUSTES

#### 2.1 Data Frame 2010

In [5]:
df_2010_raw = get_data(year = 2010, apply_format=False)
df_2010 = get_data(year = 2010, apply_format=True)

In [6]:
df_2010_raw.head(3)

,PERIODO,ORIGENFECHA,ORIGENESTACIONID,ORIGENNOMBRE,DESTINOFECHA,DESTINOESTACIONID,DESTINONOMBRE,TIEMPOUSO
0,2010,30/12/2010 19:39,6,DERECHO,30/12/2010 19:46,3,RETIRO,7.0
1,2010,30/12/2010 19:34,5,ADUANA,30/12/2010 19:47,3,RETIRO,13.0
2,2010,30/12/2010 19:10,7,PLAZA ROMA,30/12/2010 19:17,3,RETIRO,7.0


In [7]:
df_2010.head(3)

,origen_id,origen_nombre,destino_id,destino_nombre,tiempo_uso(s)
ORIGENFECHA,,,,,
2010-12-30 19:39:00,6,DERECHO,3,RETIRO,420.0
2010-12-30 19:34:00,5,ADUANA,3,RETIRO,780.0
2010-12-30 19:10:00,7,PLAZA ROMA,3,RETIRO,420.0


#### 2.2 Data Frame 2011

In [8]:
df_2011_raw = get_data(year = 2011, apply_format=False)
df_2011 = get_data(year = 2011, apply_format=True)

In [9]:
df_2011_raw.head(3)

,PERIODO,ORIGENFECHA,ORIGENESTACIONID,NOMBREORIGEN,DESTINOFECHA,DESTINOESTACIONID,NOMBREDESTINO,TIEMPOUSO
0,2011,29/09/2011 19:40,16,PARQUE LAS HERAS,29/09/2011 19:49,21,PACIFICO,10.0
1,2011,29/09/2011 19:37,3,RETIRO,29/09/2011 19:57,8,PLAZA ITALIA,20.0
2,2011,29/09/2011 19:36,19,PLAZA VICENTE LOPEZ,29/09/2011 19:41,6,DERECHO,5.0


In [10]:
df_2011.head(3)

,origen_id,origen_nombre,destino_id,destino_nombre,tiempo_uso(s)
ORIGENFECHA,,,,,
2011-09-29 19:40:00,16,PARQUE LAS HERAS,21,PACIFICO,600.0
2011-09-29 19:37:00,3,RETIRO,8,PLAZA ITALIA,1200.0
2011-09-29 19:36:00,19,PLAZA VICENTE LOPEZ,6,DERECHO,300.0


#### 2.2 Data Frame 2012

In [11]:
df_2012_raw = get_data(year = 2012, apply_format=False)

In [12]:
df_2012_raw.head(3)

,PERIODO,ORIGENFECHA,ORIGENESTACIONID,ORIGENNOMBRE,DESTINOFECHA,DESTINOESTACIONID,DESTINONOMBRE,TIEMPOUSO
0,2012,2012-01-02 07:42:15.2,28,PZA. SAN MARTIN,2012-01-02 08:10:46.89,19,PLAZA VICENTE LOPEZ,29.0
1,2012,2012-01-02 07:58:00.983,20,ONCE,2012-01-02 08:16:31.84,22,VIRREY CEVALLOS,19.0
2,2012,2012-01-02 07:59:55.59,16,PARQUE LAS HERAS,2012-01-02 08:25:56.7,7,PZA. ROMA,26.0


In [13]:
df_2012 = get_data(year = 2012, apply_format=True)

C:\Users\USUARIO\OneDrive\Documentos\3_gitHubProjects\CABA_MyT_BicicletasPublicas\notebooks\localpacks\data.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['TIEMPOUSO'] = df['TIEMPOUSO'].astype('int64', errors='ignore') * 60 #convertimos minutos(m) a segundos(s)


In [14]:
df_2012.head(3)

,origen_id,origen_nombre,destino_id,destino_nombre,tiempo_uso(s)
ORIGENFECHA,,,,,
2012-01-02 07:58:00.983,20,ONCE,22,VIRREY CEVALLOS,1140.0
2012-01-02 07:59:55.590,16,PARQUE LAS HERAS,7,PZA. ROMA,1560.0
2012-01-02 08:01:58.233,27,INDEPENDENCIA,3,RETIRO,1020.0


#### 2.3 Data Frame 2013

In [15]:
df_2013_raw = get_data(year = 2013, apply_format=False)
df_2013 = get_data(year = 2013, apply_format=True)

In [16]:
df_2013_raw.head(3)

,ID,NOMBRE_ORIGEN,ORIGEN_FECHA,DESTINO_ESTACION,DESTINO_FECHA
0,36680,ONCE,2013-01-02 07:49:52.937,PARQUE PATRICIOS,2013-01-02 08:11:36.670
1,68894,ONCE,2013-01-02 07:52:11.530,PZA. ROMA,2013-01-02 08:23:01.123
2,69014,PLAZA ALMAGRO,2013-01-02 07:53:56.140,PLAZA VICENTE LOPEZ,2013-01-02 08:25:04.297


In [17]:
df_2013.head(3)

,usuario_id,origen_nombre,destino_nombre,tiempo_uso(s)
ORIGEN_FECHA,,,,
2013-01-02 07:49:52.937,36680,ONCE,PARQUE PATRICIOS,1303
2013-01-02 07:52:11.530,68894,ONCE,PZA. ROMA,1849
2013-01-02 07:53:56.140,69014,PLAZA ALMAGRO,PLAZA VICENTE LOPEZ,1868


#### 2.3 Data Frame 2014

In [18]:
df_2014_raw = get_data(year = 2014, apply_format=False)
df_2014 = get_data(year = 2014, apply_format=True)

In [19]:
df_2014_raw.head(3)

,ID,NOMBRE_ORIGEN,ORIGEN_FECHA,DESTINO_ESTACION,DESTINO_FECHA
0,36680,ONCE,2014-01-02 08:03:44.437,SUIPACHA,2014-01-02 08:20:50.903
1,36680,SUIPACHA,2014-01-02 08:33:59.873,ALSINA,2014-01-02 08:45:23.983
2,91345,CONGRESO,2014-01-02 08:43:09.263,CONGRESO,2014-01-02 09:30:12.043


In [20]:
df_2014.head(3)

,usuario_id,origen_nombre,destino_nombre,tiempo_uso(s)
ORIGEN_FECHA,,,,
2014-01-02 08:03:44.437,36680,ONCE,SUIPACHA,1026
2014-01-02 08:33:59.873,36680,SUIPACHA,ALSINA,684
2014-01-02 08:43:09.263,91345,CONGRESO,CONGRESO,2822


#### 2.4 Data Frame 2015

In [21]:
df_2015_raw = get_data(year = 2015, apply_format=False)
df_2015 = get_data(year = 2015, apply_format=True)

In [22]:
df_2015_raw.head(3)

,periodo,genero,fecha_hora_retiro,origen_estacion,nombre_origen,destino_estacion,nombre_destino,tiempo_uso
0,2015,M,01/05/2015 00:00:18,25,Plaza Güemes,29,Parque Centenario,0h 26min 24seg
1,2015,F,01/05/2015 00:02:06,17,Plaza Almagro,25,Plaza Güemes,0h 57min 53seg
2,2015,M,01/05/2015 00:02:53,17,Plaza Almagro,25,Plaza Güemes,1h 3min 8seg


In [23]:
df_2015.head(3)

,genero,origen_id,origen_nombre,destino_id,destino_nombre,tiempo_uso(s)
fecha_hora_retiro,,,,,,
2015-05-01 00:00:18,M,25,Plaza Güemes,29,Parque Centenario,1584.0
2015-05-01 00:02:06,F,17,Plaza Almagro,25,Plaza Güemes,3473.0
2015-05-01 00:02:53,M,17,Plaza Almagro,25,Plaza Güemes,3788.0


#### 2.5 Data Frame 2016

In [24]:
df_2016_raw = get_data(year = 2016, apply_format=False)
df_2016 = get_data(year = 2016, apply_format=True)

In [25]:
df_2016_raw.head(3)

,periodo,genero,fecha_hora_retiro,origen_estacion,nombre_origen,destino_estacion,nombre_destino,tiempo_uso
0,2016,M,01/01/2016 02:21:41,11,Tribunales,5,Plaza Italia,0h 16min 29seg
1,2016,M,01/01/2016 08:38:19,28,Plaza Boedo,28,Plaza Boedo,0h 55min 26seg
2,2016,M,01/01/2016 15:01:07,28,Plaza Boedo,30,Peña,0h 40min 27seg


In [26]:
df_2016.head(3)

,genero,origen_id,origen_nombre,destino_id,destino_nombre,tiempo_uso(s)
fecha_hora_retiro,,,,,,
2016-01-01 02:21:41,M,11,Tribunales,5,Plaza Italia,989.0
2016-01-01 08:38:19,M,28,Plaza Boedo,28,Plaza Boedo,3326.0
2016-01-01 15:01:07,M,28,Plaza Boedo,30,Peña,2427.0


#### 2.6 Data Frame 2017

In [27]:
df_2017_raw = get_data(year=2017, apply_format=False)
df_2017 = get_data(year=2017, apply_format=True)

In [28]:
df_2017_raw.head(3)

,periodo,genero,fecha_hora_retiro,origen_estacion,nombre_origen,destino_estacion,nombre_destino,tiempo_uso
0,2017,M,01/01/2017 00:02:14,60,25 de Mayo,4,Plaza Roma,0h 27min 44seg
1,2017,M,01/01/2017 00:02:59,79,Azucena Villaflor,54,Acuña de Figueroa,1h 57min 21seg
2,2017,M,01/01/2017 00:07:13,11,Tribunales,84,Lavalle,0h 25min 8seg


In [29]:
df_2017.head(3)

,genero,origen_id,origen_nombre,destino_id,destino_nombre,tiempo_uso(s)
fecha_hora_retiro,,,,,,
2017-01-01 00:02:14,M,60,25 de Mayo,4,Plaza Roma,1664.0
2017-01-01 00:02:59,M,79,Azucena Villaflor,54,Acuña de Figueroa,7041.0
2017-01-01 00:07:13,M,11,Tribunales,84,Lavalle,1508.0


#### 2.7 Data Frame 2018

In [30]:
df_2018_raw = get_data(year=2018, apply_format=False)
df_2018 = get_data(year=2018, apply_format=True)

In [31]:
df_2018_raw.head(3)

,id_usuario,bici_fecha_hora_retiro,bici_tiempo_uso,bici_nombre_estacion_origen,bici_estacion_origen,bici_nombre_estacion_destino,bici_estacion_destino,bici_sexo,bici_edad
0,5453,2018-01-01 00:08:05,00:19:53,Uruguay,45,Virrey Cevallos,183,M,45
1,673,2018-01-01 00:18:05,00:26:19,Posadas,189,Guardia Vieja,110,M,61
2,179119,2018-01-01 00:20:14,00:27:39,Hospital Rivadavia,50,Padilla,31,F,52


In [32]:
df_2018.head(3)

,usuario_id,origen_nombre,origen_id,destino_nombre,destino_id,usuario_genero,usario_edad,tiempo_uso(s)
bici_fecha_hora_retiro,,,,,,,,
2018-01-01 00:08:05,5453,Uruguay,45,Virrey Cevallos,183,M,45,1193
2018-01-01 00:18:05,673,Posadas,189,Guardia Vieja,110,M,61,1579
2018-01-01 00:20:14,179119,Hospital Rivadavia,50,Padilla,31,F,52,1659
